# 02 – V1 MLP Baseline (Flattened Pixels)

Goal : train a simple multilayer-perceptron on 64 × 64 RGB images  
to produce a reproducible baseline that should **beat the 66 % majority-class reference** measured in notebook 01.



## 1  Load & preprocess images
*Replace the folder paths with your real dataset directories before running.*


In [ ]:
from pathlib import Path
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from src.data_loader import load_images

# --- CONFIG ---
folder_paths = [
    "../data/sample_images/recyclable",        # TODO replace
    "../data/sample_images/non_recyclable"     # TODO replace
]
class_names  = ["recyclable", "non-recyclable"]
target_size  = (64, 64)

# --- LOAD ---
X, y, ignored = load_images(folder_paths, class_names, target_size)
print(f"Loaded {X.shape[0]} images  |  ignored files {ignored}")

# --- ENCODE labels ---
le   = LabelEncoder()
y_int  = le.fit_transform(y)
y_hot  = to_categorical(y_int, num_classes=len(le.classes_))

# --- SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y_hot, test_size=0.2, random_state=42, stratify=y_hot)

print("Train:", X_train.shape, "Test:", X_test.shape)


### Flatten & scale to [0-1]

The MLP expects 1-D feature vectors.  
We simply divide by 255 so each pixel channel is in [0, 1].


In [ ]:
# flatten
X_train_f = X_train.reshape(len(X_train), -1).astype("float32") / 255.0
X_test_f  = X_test .reshape(len(X_test ), -1).astype("float32") / 255.0
print("Flattened dim =", X_train_f.shape[1])


## 2  Build & compile the MLP


In [ ]:
from src.model_mlp import build_mlp
from src.compile_utils import compile_model, early_stop

mlp = build_mlp(input_dim=X_train_f.shape[1], classes=y_train.shape[1])
mlp = compile_model(mlp, lr=1e-3, loss="categorical_crossentropy")
mlp.summary()


## 3  Train
EarlyStopping halts training if validation loss stops improving for 3 epochs.


In [ ]:
H = mlp.fit(
    X_train_f, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    callbacks=[early_stop(patience=3)],
    verbose=2
)


## 4  Learning curves


In [ ]:
from src.plotting import plot_history
plot_history(H)


## 5  Evaluation on hold-out test set


In [ ]:
from src.evaluation import evaluate
evaluate(mlp, X_test_f, y_test, labels=le.classes_)


## 6  Discussion

* **Accuracy** ≈ … (should exceed the 66 % baseline).  
* Class-wise precision/recall shows …  
* Overfitting? Validation curve diverges after epoch …  

This MLP is still **image-naïve** (no spatial structure).  
Next notebook (03) upgrades to a small **CNN** to exploit local features.
